# WandbCallbackHandler Demo

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv
load_dotenv("/Users/ayushthakur/integrations/llamaindex/apis.env")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [20]:
from llama_index.callbacks import CallbackManager, CBEventType
from llama_index.callbacks import LlamaDebugHandler, WandbCallbackHandler
from llama_index import (
    GPTListIndex, GPTTreeIndex, GPTVectorStoreIndex,
    ServiceContext, SimpleDirectoryReader, LLMPredictor,
    GPTSimpleKeywordTableIndex
)
from llama_index.indices.composability import ComposableGraph
from langchain.chat_models import ChatOpenAI

## Setup LLM

In [7]:
llm_predictor = LLMPredictor(llm=ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0))

## W&B Callback Manager Setup

In [8]:
llama_debug = LlamaDebugHandler(print_trace_on_end=True)

# wandb.init args
run_args = dict(
    project="llamaindex-demo",
)

wandb_callback = WandbCallbackHandler(run_args=run_args)

callback_manager = CallbackManager([llama_debug, wandb_callback])
service_context = ServiceContext.from_defaults(callback_manager=callback_manager, llm_predictor=llm_predictor)

wandb: Streaming LlamaIndex events to W&B at https://wandb.ai/ayut/llamaindex-demo/runs/an4z3ut9
wandb: `WandbCallbackHandler` is currently in beta.
wandb: Please report any issues to https://github.com/wandb/wandb/issues with the tag `llamaindex`.


> After running the above cell, you will get the W&B run page URL. Here you will find a trace table with all the events tracked using [Weights and Biases' Prompts](https://docs.wandb.ai/guides/prompts) feature.

## 1. Indexing

In [9]:
docs = SimpleDirectoryReader("../data/paul_graham/").load_data()

In [11]:
index = GPTVectorStoreIndex.from_documents(docs, service_context=service_context)
query_engine = index.as_query_engine()

**********
Trace: index_construction
    |_node_parsing ->  0.099798 seconds
      |_chunking ->  0.099281 seconds
    |_embedding ->  1.408279 seconds
    |_embedding ->  1.418708 seconds
    |_embedding ->  0.998625 seconds
**********


wandb: WARNING Serializing object of type str that is 150112 bytes
wandb: WARNING Serializing object of type str that is 150112 bytes


## 2. Query Over Index

In [12]:
response = query_engine.query("What did the author do growing up?")
print(response, sep="\n")

**********
Trace: query
    |_query ->  6.620803 seconds
      |_retrieve ->  0.934077 seconds
        |_embedding ->  0.922523 seconds
      |_synthesize ->  5.686477 seconds
        |_llm ->  5.668209 seconds
**********
The author worked on writing and programming before college, specifically writing short stories and programming on an IBM 1401 using an early version of Fortran. They later got a TRS-80 and wrote simple games and a word processor.


## 3. Build Complex Indices

In [13]:
# fetch "New York City" page from Wikipedia
from pathlib import Path

import requests
response = requests.get(
    'https://en.wikipedia.org/w/api.php',
    params={
        'action': 'query',
        'format': 'json',
        'titles': 'New York City',
        'prop': 'extracts',
        'explaintext': True,
    }
).json()
page = next(iter(response['query']['pages'].values()))
nyc_text = page['extract']

data_path = Path('data')
if not data_path.exists():
    Path.mkdir(data_path)

with open('data/nyc_text.txt', 'w') as fp:
    fp.write(nyc_text)

In [14]:
# load NYC dataset
nyc_documents = SimpleDirectoryReader('data/').load_data()
# load PG's essay
essay_documents = SimpleDirectoryReader('../paul_graham_essay/data/').load_data()

In [15]:
# build NYC index
nyc_index = GPTVectorStoreIndex.from_documents(nyc_documents, service_context=service_context)

**********
Trace: index_construction
    |_node_parsing ->  0.167444 seconds
      |_chunking ->  0.16678 seconds
    |_embedding ->  1.621093 seconds
    |_embedding ->  1.092478 seconds
    |_embedding ->  1.298416 seconds
    |_embedding ->  1.325762 seconds
**********


wandb: WARNING Serializing object of type str that is 261494 bytes
wandb: WARNING Serializing object of type str that is 261494 bytes


In [16]:
# build essay index
essay_index = GPTVectorStoreIndex.from_documents(essay_documents, service_context=service_context)

**********
Trace: index_construction
    |_node_parsing ->  2.4e-05 seconds
**********


## 4. Query Over Graph Index

In [17]:
nyc_index_summary = """
    New York, often called New York City or NYC, 
    is the most populous city in the United States. 
    With a 2020 population of 8,804,190 distributed over 300.46 square miles (778.2 km2), 
    New York City is also the most densely populated major city in the United States, 
    and is more than twice as populous as second-place Los Angeles. 
    New York City lies at the southern tip of New York State, and 
    constitutes the geographical and demographic center of both the 
    Northeast megalopolis and the New York metropolitan area, the 
    largest metropolitan area in the world by urban landmass.[8] With over 
    20.1 million people in its metropolitan statistical area and 23.5 million 
    in its combined statistical area as of 2020, New York is one of the world's 
    most populous megacities, and over 58 million people live within 250 mi (400 km) of 
    the city. New York City is a global cultural, financial, and media center with 
    a significant influence on commerce, health care and life sciences, entertainment, 
    research, technology, education, politics, tourism, dining, art, fashion, and sports. 
    Home to the headquarters of the United Nations, 
    New York is an important center for international diplomacy,
    an established safe haven for global investors, and is sometimes described as the capital of the world.
"""
essay_index_summary = """
    Author: Paul Graham. 
    The author grew up painting and writing essays. 
    He wrote a book on Lisp and did freelance Lisp hacking work to support himself. 
    He also became the de facto studio assistant for Idelle Weber, an early photorealist painter. 
    He eventually had the idea to start a company to put art galleries online, but the idea was unsuccessful. 
    He then had the idea to write software to build online stores, which became the basis for his successful company, Viaweb. 
    After Viaweb was acquired by Yahoo!, the author returned to painting and started writing essays online. 
    He wrote a book of essays, Hackers & Painters, and worked on spam filters. 
    He also bought a building in Cambridge to use as an office. 
    He then had the idea to start Y Combinator, an investment firm that would 
    make a larger number of smaller investments and help founders remain as CEO. 
    He and his partner Jessica Livingston ran Y Combinator and funded a batch of startups twice a year. 
    He also continued to write essays, cook for groups of friends, and explore the concept of invented vs discovered in software. 
"""

In [21]:
graph = ComposableGraph.from_indices(
    GPTSimpleKeywordTableIndex,
    [nyc_index, essay_index], 
    index_summaries=[nyc_index_summary, essay_index_summary],
    max_keywords_per_chunk=50,
    service_context=service_context
)

**********
Trace: graph_construction
**********


In [22]:
query_engine = graph.as_query_engine()
response = query_engine.query(
    "What is the climate of New York City like? How cold is it during the winter?", 
)

**********
Trace: query
    |_query ->  23.911113 seconds
      |_retrieve ->  2.511374 seconds
        |_llm ->  2.509327 seconds
      |_query ->  9.732959 seconds
        |_retrieve ->  0.535612 seconds
          |_embedding ->  0.510166 seconds
        |_synthesize ->  9.196923 seconds
          |_synthesize ->  9.196816 seconds
            |_llm ->  9.177568 seconds
      |_synthesize ->  11.666183 seconds
        |_synthesize ->  11.666062 seconds
          |_llm ->  11.659629 seconds
**********


### Close W&B Callback Handler

When we are done tracking our events we can close the wandb run.

In [23]:
wandb_callback.finish()